---
title: Data Cleaning
format:
  html:
    css: "styles.css"
    embed-resources: true
    code-fold: true
---

## Data Cleaning

This tab aims to clean the data sets that will be used in all the others. The goal is to have our data sets ready to be used in the analysis. Before doing so, we will explain some common processes that are done or checked by most data scientists. However, it is worth mentioning that data cleaning is not a linear process, and it is not always possible to follow a specific order. In fact, it is common to go back and forth between the different steps.

### Importing and visualizing the data

The first most common step is to import and visualize how the data looks like. This is shown already in our data gathering tab by displaying the top 4 or 5 rows. It is also common to display the number of columns and rows that the data has to give us a sense of the size of the data set. In addition, it is important to check the names the data types of each column in order to see whether it will be beneficial to change them for your use (depending on the type of analysis you will do).

### Dealing with missing values

Dealing with missing values can be complicated. There is not a right or wrong way to do it. However, the goal should be to deal with them in a way that the data-set integrity is mantained while being able to properly perform and assess the analysis intended. The most important thing is to record the process of dealing with missing values and to explain why you chose to do it in that way. This is important because it will allow others to understand your process and to replicate it if needed while also reporting the limitations of your analysis due to those missing values.

### Dealing with outliers

Another common practice is to deal with outliers. If you detect outliers, you should decide whether to keep them or not. This depends on the type of analysis that you do and will also depend on the type of data that you have.

### Dealing with duplicates

It is also important to check for duplicates. You do not want to count a data point twice just because of an administrative error. If you do so, you will give more importance to that data point than it should have.

### Our process

For these three data sets, we will cheack for NaN values and select the columns (variables) that we are going to use. These data sets do not hold duplicates. We want to keep outliers for our tests due to the fact that they may be important for our analysis. However, in each tab, this will be crosschecked. Additiopnally, there are no diuplicates in our data sets.

## Information about US census blocks dataset

The followin code attempts to clean the EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv file. It keeps the columns that will be relevant for the analysis and clustering of the census block data of the US cities. The columns excluded are all the administrative variables, the individualistic columns such as IDs that would not allow us to perform clustering properly, and other columns that are not relevant for the analysis (all columns droped are found under the comment "# Drop unwanted columns" in the code chunk, and their definitions inside EPA pdf inside the data folder).

All variables have been checked and there are no missing values that would affect our study.

In [ ]:
import pandas as pd

file_path = '../../data/Raw_Data_project_Pub.Transport_5000/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv'

df = pd.read_csv(file_path)

# Drop unwanted columns
df.drop(columns=["OBJECTID", "GEOID10", "GEOID20", "STATEFP", "COUNTYFP", "COUNTYFP", "TRACTCE", "BLKGRPCE", "CSA", "CSA_Name", "CBSA", "CBSA_Name", "CountHU", "HH", "D1A", "D1C8_OFF", "D4D", "D4E", "D2A_JPHH", "D2B_E5MIX", "D2B_E5MIXA", "D2B_E8MIX","D2B_E8MIXA","D2A_EPHHM","D2C_TRPMX1","D2C_TRPMX2","D2C_TRIPEQ","D2R_JOBPOP","D2R_WRKEMP","D2A_WRKEMP","D2C_WREMLX","D4A","D4B025","D4B050","D4C","D5AR","D5AE","D5BR","D5BE","D5CR","D5CRI","D5CE","D5CEI","D5DR","D5DRI","D5DE","D5DEI"], inplace=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(df.isna().sum())

df.to_csv('../../data/Clean_Data_project_Pub.Transport_5000/EPA_SmartLocationDatabase_V3_Jan_2021_Clean.csv', index=False)

## Text data 

The following code cleans our text data file top_pub_transp_urls.json. It removes the escape sequences, non-alphabetic characters, the extra white spaces and strips the text. Then, it saves it in the clean data folder as cleaned_text_data.json (it can be found inside the reddit data on the raw data folder, but also inside the clean data folder).

Note: the sentiment analysis was already extracted and saved in the data tab.

In [ ]:
import json
import re

with open('top_pub_transp_urls.json', 'r') as json_file:
    data = json.load(json_file)

def clean_text(text):
    cleaned_text = re.sub(r'\\u....', '', text) 
    cleaned_text = re.sub(r'[^A-Za-z\s]', ' ', cleaned_text) 
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) 
    cleaned_text = cleaned_text.strip() 
    return cleaned_text

cleaned_text_data = []

for item in data:
    cleaned_text = clean_text(item.get("text", ""))
    cleaned_text_data.append({"url": item["url"], "cleaned_text": cleaned_text})

with open('cleaned_text_data.json', 'w') as json_file:
    json.dump(cleaned_text_data, json_file, indent=4)

#print("Text data cleaned and saved to cleaned_text_data.json.")

## Fuel and Energy data
The following code selects the columns that are relevant for the analysis and drops the rest. Since this data set will be used for modeling purposes, the redundant columns that were directly connected to the target variable have been dropped (in this case, since we will focuss on predicting avg miles per galon of fossil fuels, all other information of usage of fossil fuels is being dropped). It also creates a new column that averages the mpg between diesel and gasoline, and then drops all the rows that have missing values in this new  (since they will not be able to be used for our modeling purposes). The final data set is saved in the clean data folder as 2021_Fuel_and_Energy_mpg_fossil_fuels.csv.

Note: the columns kept are: City, Agency, Organization Type, Primary UZA Population, Mode, TOS, Electric Battery, and the new column that averages the mpg between diesel and gasoline (called avg_mpg_fossil_fuels).

In [ ]:
import pandas as pd

file_path = '../../data/Raw_Data_project_Pub.Transport_5000/2021_Fuel_and Energy.xlsm'

df = pd.read_excel(file_path, sheet_name='Fuel and Energy')

columns_to_keep = ['City', 'Agency', 'Organization Type', 'Primary UZA Population', 'Mode', 'TOS', 'Electric Battery', 'Diesel (mpg)', 'Gasoline (mpg)']

# Keep only the specified columns
df_filtered = df[columns_to_keep]

df_filtered['avg_mpg_fossil_fuels'] = df_filtered[['Diesel (mpg)', 'Gasoline (mpg)']].mean(axis=1, skipna=True)

df_filtered = df_filtered.dropna(subset=['avg_mpg_fossil_fuels'])

df_filtered = df_filtered.drop(['Diesel (mpg)', 'Gasoline (mpg)'], axis=1)

df_filtered.to_csv('../../data/Clean_Data_project_Pub.Transport_5000/2021_Fuel_and_Energy_mpg_fossil_fuels.csv', index=False)

## Border Crossing Entry Data

The following code cleans the data of Border_Crossing_Entry_Data_20231103.csv (found in the raw data folder). Since this data set will be used for modeling purposes (in the Naive Bayes tab), we will use the columns: Value, State, Measure. The cleaning code will only select this columns and save them into a new csv called Clean_Border_Crossing_Entry_Data_20231103.csv in the clean data folder.

Note: NaN values have been checked and there are none

In [4]:
import pandas as pd

file_path = '../../data/Raw_Data_project_Pub.Transport_5000/Border_Crossing_Entry_Data_20231103.csv'
df = pd.read_csv(file_path)

columns_to_keep = ["Value", "State", "Measure"]

df=df[columns_to_keep]

df.isna().sum()

df.to_csv('../../data/Clean_Data_project_Pub.Transport_5000/Clean_Border_Crossing_Entry_Data_20231103.csv', index=False)